In [1]:
import os
import pandas as pd
import numpy as np

from math import sqrt
from sklearn import preprocessing

from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense

import tensorflow as tf
import matplotlib.pyplot as plt
import datetime

### Load the dataset

In [2]:
"""#### Loading the data"""

orig_data = pd.read_csv('./Data/UCI-PM.csv')

# Drop unnecessary columns
orig_data = orig_data.drop(columns=['cbwd', 'No'])

# Drop na values
aug_data = orig_data.dropna()
aug_data.describe()

,DEWP,TEMP,PRES,Iws,Is,Ir,pm2.5,pm2.5_pre
count,41757.000000,41757.000000,41757.000000,41757.000000,41757.000000,41757.000000,41757.000000,41757.000000
mean,1.750174,12.401561,1016.442896,23.866747,0.055344,0.194866,98.613215,98.613215
std,14.433658,12.175215,10.300733,49.617495,0.778875,1.418165,92.050387,92.050387
min,-40.000000,-19.000000,991.000000,0.450000,0.000000,0.000000,0.000000,0.000000
25%,-10.000000,2.000000,1008.000000,1.790000,0.000000,0.000000,29.000000,29.000000
50%,2.000000,14.000000,1016.000000,5.370000,0.000000,0.000000,72.000000,72.000000
75%,15.000000,23.000000,1025.000000,21.910000,0.000000,0.000000,137.000000,137.000000
max,28.000000,42.000000,1046.000000,565.490000,27.000000,36.000000,994.000000,994.000000


#### Normalize and function to inverse transform the data

In [3]:
"""#### Scaling and inverse_transform method"""

# Scale the whole data
temp_x = aug_data.values # Returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
temp_x_scaled = min_max_scaler.fit_transform(temp_x)
df = pd.DataFrame(temp_x_scaled, columns = aug_data.columns)

# Inverse_transform of PM2.5
pm_inv_scaler = preprocessing.MinMaxScaler()
temp_scaler = np.reshape(aug_data['pm2.5'].values, (-1,1))
pm_inv_scaler.fit(temp_scaler)

MinMaxScaler()

In [4]:
def inverse_pm_scaler(data):
    data = np.asarray(data)
    data = np.reshape(data, (-1, 1))
    return pm_inv_scaler.inverse_transform(data)

#### Model Utilities

In [5]:
# Path to save the generated files locally
path = "./msvlstm/"

In [6]:
# Convert data to supervised learning format
def convert_to_supervised(sequences, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out-1
        # check if we are beyond the dataset
        
        if out_end_ix > len(sequences):
            break
        
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1:out_end_ix, -1]
        X.append(seq_x)
        y.append(seq_y)
    return np.asarray(X), np.asarray(y)

In [7]:
# Split data into test and train
def split_data(x, y, split_percentage = 0.8):
    n_train_hours = (int)(split_percentage * len(x))

    train_x = x[:n_train_hours, :]
    train_y = y[:n_train_hours]

    test_x  = x[n_train_hours:, :]
    test_y = y[n_train_hours:]

    return train_x, train_y, test_x, test_y

#### Compute Error matrices

In [8]:
def rmse_metric(actual, predicted):
    rmse = []
    for i in range(len(actual[1])):
        c_actual = inverse_pm_scaler(actual[:, i])
        c_predicted = inverse_pm_scaler(predicted[:, i])
        sum_error = 0.0
        for j in range(len(c_actual)):
            prediction_error = c_predicted[j] - c_actual[j]
            sum_error += (prediction_error ** 2)
        mean_error = sum_error / float(len(c_actual))
        rmse.append(sqrt(mean_error))

    return rmse

In [9]:
def mape_function(actual, predicted):
    mape = []
    for i in range(len(actual[1])):
        c_actual = inverse_pm_scaler(actual[:, i])
        c_predicted = inverse_pm_scaler(predicted[:, i])
        sum_error = 0.0

        for j in range(len(c_actual)):
            # Avoiding divide by zero
            if c_actual[j] == 0:
                c_actual[j] = 1
            prediction_error = np.abs((c_actual[j] - c_predicted[j])/c_actual[j])
            sum_error += prediction_error
        
        mape_error = sum_error / float(len(c_actual))*100
        mape.append(mape_error[0])
    
    return mape

#### Define Model Architecture

In [10]:
def vanilla_lstm(x, y, step, nodes, epoch, n_features=7):
    model_id = "msvlstm_" + str(step) + "_" + str(nodes) + "_" + str(epoch) + '/'

    # define model
    base_model = Sequential()
    base_model.add(LSTM(nodes, activation='relu', return_sequences=True, input_shape=(step, n_features)))
    base_model.add(LSTM(nodes, activation='relu'))
    base_model.add(Dense(6))  # Forecasting 6 steps in ahead.
    base_model.compile(optimizer='adam', loss='mse')

    # Tensorboard
    v_log_dir = path + "tb_vlstm/" + model_id + "/logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    vlstm_tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=v_log_dir, histogram_freq=1)

    # fit base model
    history = base_model.fit(x, y, callbacks=[vlstm_tensorboard_callback], validation_split=0.2, epochs=epoch, verbose=0)

    return base_model, history

#### Grid-Search Method

In [ ]:
def grid_vlstm():
    # choose a number of time steps
    steps_out = 6

    steps_in = [2, 4, 6, 12, 24]
    epochs = [100]
    n_nodes = [32, 64, 128, 256]

    temp = []
        for step in steps_in:
            x, y = convert_to_supervised(df.values, step, steps_out)
            train_x, train_y, test_x, test_y = split_data(x, y)

        for epoch in epochs:
            for nodes in n_nodes:
                model_name = "msvlstm_" + str(step) + "_" + str(nodes) + "_" + str(epoch)

                # Call method to fit the model
                v_model, history = vanilla_lstm(train_x, train_y, step, nodes, epoch)

                # Calculate Train RMSE
                train_yhat = v_model.predict(train_x, verbose=0)

                train_rmse = rmse_metric(train_y, train_yhat)
                train_mape = mape_function(train_y, train_yhat)

                # Calculate Test RMSE
                test_yhat = v_model.predict(test_x, verbose=0)

                test_rmse = rmse_metric(test_y, test_yhat)
                test_mape = mape_function(train_y, train_yhat)

                temp.append([step, nodes, epoch, train_rmse, train_mape, test_rmse, train_mape])

                # Save all the data locally
                os.makedirs(os.path.dirname(path + 'train_y/'), exist_ok=True)
                os.makedirs(os.path.dirname(path + 'train_yhat/'), exist_ok=True)
                os.makedirs(os.path.dirname(path + 'test_y/'), exist_ok=True)
                os.makedirs(os.path.dirname(path + 'test_yhat/'), exist_ok=True)
                os.makedirs(os.path.dirname(path + 'history/'), exist_ok=True)
                os.makedirs(os.path.dirname(path + 'model/'), exist_ok=True)

                np.save(path + 'train_y/' + model_name, test_y)
                np.save(path + 'train_yhat/' + model_name, test_yhat)
                np.save(path + 'test_y/' + model_name, test_y)
                np.save(path + 'test_yhat/' + model_name, test_yhat)
                np.save(path + 'history/' + model_name, history.history)
                v_model.save(path + 'model/' + model_name + '.h5')

    temp_df = pd.DataFrame(temp, columns=['Step Size', 'No of Nodes', 'Epoch', 'Train RMSE', 'Train MAPE', 'Test RMSE', 'Train MAPE'])
    temp_df.to_csv(path + 'msvanilla.csv', index=False)


### Call the grid-search method and train the models

In [ ]:
# The model will take quite a long time based on system configuration.
grid_vlstm()